In [1]:
"""
CSIRO Biomass Competition - Two-Stream Three-Head Regression Model with Visualization
================================================================================
This script implements a pipeline for predicting biomass from images with comprehensive visualization.

Pipeline Overview:
1. Data Preparation: Load CSV → Pivot → Stratified K-Fold split
2. Preprocessing: Image split (left/right) → Augmentation → Normalization
3. Model: Shared Backbone → Feature concatenation → 3 dedicated heads
4. Training: Two-stage learning (Freeze→Unfreeze) → Weighted loss → R² evaluation
5. Visualization: Learning curves, Fold comparison, Prediction scatter plots
"""
from __future__ import annotations
from dataclasses import dataclass, field
from pathlib import Path
from typing import Callable, Optional
import os
import gc
import time
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, LambdaLR
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm
import cv2
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
torch.__version__

'2.6.0+cu124'

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True, warn_only=True) 
    print(f"Global seed set to {seed}")
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
seed_everything(42)

Global seed set to 42


In [3]:
print(timm.__version__)
print(len(timm.list_models("*dinov2*")))
print(timm.list_models("*dinov2*"))
print(timm.list_models("*dinov3*"))

1.0.22
8
['vit_base_patch14_dinov2', 'vit_base_patch14_reg4_dinov2', 'vit_giant_patch14_dinov2', 'vit_giant_patch14_reg4_dinov2', 'vit_large_patch14_dinov2', 'vit_large_patch14_reg4_dinov2', 'vit_small_patch14_dinov2', 'vit_small_patch14_reg4_dinov2']
['vit_7b_patch16_dinov3', 'vit_base_patch16_dinov3', 'vit_base_patch16_dinov3_qkvb', 'vit_huge_plus_patch16_dinov3', 'vit_huge_plus_patch16_dinov3_qkvb', 'vit_large_patch16_dinov3', 'vit_large_patch16_dinov3_qkvb', 'vit_small_patch16_dinov3', 'vit_small_patch16_dinov3_qkvb', 'vit_small_plus_patch16_dinov3', 'vit_small_plus_patch16_dinov3_qkvb']


In [4]:
# ============================================================================
# Configuration Management
# ============================================================================

@dataclass
class Config:
    """
    Data class for managing pipeline-wide configuration.
    
    Centralizing all hyperparameters and constants ensures
    experiment reproducibility and facilitates configuration changes.
    """
    
    # Path settings
    base_path: Path = Path('csiro-biomass')
    train_csv: Path = field(init=False)
    image_dir: Path = field(init=False)
    output_dir: Path = Path('baseline/')  # Directory for saving visualizations
    
    # Model settings
    # model_name: str = 'convnext_small.fb_in22k' 
    model_name: str = 'eva02_base_patch14_448.mim_in22k_ft_in1k'
    # model_name: str = 'efficientnet_b4'
    pretrained: bool = True
    img_size: int = 448
    
    # Device settings
    device: torch.device = field(default_factory=lambda: torch.device(
        'cuda' if torch.cuda.is_available() else 'cpu'
    ))
    
    # Training settings
    batch_size: int = 12 
    epochs: int = 50
    learning_rate: float = 3e-4  # Learning rate for Stage 1
    finetune_lr: float = 3e-6     # Learning rate for Stage 2
    freeze_epochs: int = 20        # Number of epochs to freeze backbone
    num_workers: int = 18
    
    # Cross-validation settings
    n_folds: int = 5
    random_state: int = 42
    cv_strategy = 'groupby_Sampling_Date'
    
    # Target settings
    # Three targets used for training (loss calculation)
    train_target_cols: list[str] = field(default_factory=lambda: [
        'Dry_Total_g', 'GDM_g', 'Dry_Green_g'
    ])
    
    # Five targets used for evaluation (R² score calculation)
    all_target_cols: list[str] = field(default_factory=lambda: [
        'Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g'
    ])
    
    # Loss function weights (corresponding to 3 training targets)
    loss_weights: dict[str, float] = field(default_factory=lambda: {
        'total_loss': 0.5,
        'gdm_loss': 0.2,
        'green_loss': 0.1
    })
    
    # R² score weights (corresponding to 5 evaluation targets)
    r2_weights: list[float] = field(default_factory=lambda: [
        0.1, 0.1, 0.1, 0.2, 0.5
    ])

    variant = 'tiled_film' # tiled_film or tiled or normal
    backbone_name = 'vit_base_patch14_reg4_dinov2'
    DINO_CANDIDATES = [
        "vit_base_patch14_dinov2",
        "vit_base_patch14_reg4_dinov2",
        "vit_large_patch16_dinov3",
        "vit_base_patch16_dinov3",
        "vit_small_patch16_dinov3",   
    ]

    DROPOUT = 0.30
    HIDDEN_RATIO = 0.25
    GRID = (2, 2)  # tiled / tiled_film
    
    def __post_init__(self) -> None:
        """Construct paths after initialization and create output directory"""
        self.train_csv = self.base_path / 'train.csv'
        self.image_dir = self.base_path / 'train'
        
        # Create output directory for visualizations
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
    def display_info(self) -> None:
        """Display configuration information"""
        print(f"{'='*70}")
        print(f"Configuration")
        print(f"{'='*70}")
        print(f"Device: {self.device}")
        print(f"Backbone: {self.model_name}")
        print(f"Image Size: {self.img_size}x{self.img_size}")
        print(f"Batch Size: {self.batch_size}")
        print(f"Total Epochs: {self.epochs}")
        print(f"  - Stage 1 (Freeze): Epoch 1-{self.freeze_epochs} (LR={self.learning_rate})")
        print(f"  - Stage 2 (Finetune): Epoch {self.freeze_epochs+1}-{self.epochs} (LR={self.finetune_lr})")
        print(f"Cross-Validation: {self.n_folds}-Fold")
        print(f"Output Directory: {self.output_dir}")
        print(f"{'='*70}\n")


# ============================================================================
# Data Preparation
# ============================================================================

class DataPreparator:
    """
    Class responsible for data loading and preprocessing.
    
    Main functions:
    - Load and pivot CSV
    - Stratified K-Fold splitting
    """
    
    def __init__(self, config: Config):
        """
        Args:
            config: Configuration object
        """
        self.config = config
        self.df_wide: Optional[pd.DataFrame] = None
        
    def load_and_pivot(self) -> pd.DataFrame:
        """
        Load CSV and convert from long format to wide format.
        
        Returns:
            Wide-format DataFrame (each row is one image, each column is one target)
            
        Why not: Using pivot() instead of pivot_table()
            → Image paths are guaranteed to be unique
        """
        print(f"Loading CSV: {self.config.train_csv}")
        
        try:
            df_long = pd.read_csv(self.config.train_csv)
            print(f"Long format: {len(df_long)} rows")
            df_long[['sample_id_prefix', 'sample_id_suffix']] = df_long.sample_id.str.split('__', expand=True)
            # Pivot transformation: image_path × target_name → values
            cols = ['sample_id_prefix', 'image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm']
            df_wide = df_long.groupby(cols).apply(lambda df: df.set_index('target_name').target)
            df_wide.reset_index(inplace=True)
            df_wide.columns.name = None
            df_wide['Sampling_Date_Month'] = df_wide.Sampling_Date.apply(lambda x: x.split('/')[1].strip())
            print(f"Wide format: {len(df_wide)} rows × {len(df_wide.columns)} columns")
            print(f"\nFirst 5 rows:\n{df_wide.head()}\n")
            
            self.df_wide = df_wide
            return df_wide
            
        except FileNotFoundError:
            print(f"Error: {self.config.train_csv} not found")
            # Return dummy DataFrame on error (prevent downstream crashes)
            return pd.DataFrame(columns=['image_path'] + self.config.all_target_cols)
    
    def create_stratified_folds(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Assign folds using StratifiedGroupKFold, then apply an optimization loop
        to balance both total sample counts and Ryegrass_Clover distribution
        across folds.
        """
        df = df.copy()
        df['fold'] = -1
        
        # Configuration
        cv_strategy = getattr(self.config, 'cv_strategy', 'groupby_Sampling_Date')
        n_folds = self.config.n_folds
        seed = self.config.random_state
        
        # 1. Initial Binning for Stratification (Dry_Total_g)
        num_bins = 4
        df['total_bin'] = pd.qcut(
            df['Dry_Total_g'], 
            q=num_bins, 
            labels=False, 
            duplicates='drop'
        )
    
        # 2. Initial Split (The Starting Point)
        if cv_strategy == 'groupby_Sampling_Date':
            print(f"Preparing {n_folds}-Fold StratifiedGroupKFold (Init)...")
            kfold = StratifiedGroupKFold(n_splits=n_folds, shuffle=True, random_state=seed)
            split_gen = kfold.split(X=df, y=df['total_bin'], groups=df['Sampling_Date'])
        else:
            # Fallback for standard KFold (no groups)
            kfold = KFold(n_splits=n_folds, shuffle=True, random_state=seed)
            split_gen = kfold.split(X=df, y=df['total_bin'])
    
        for i, (trn_idx, val_idx) in enumerate(split_gen):
            df.loc[val_idx, 'fold'] = i
    
        # ====================================================================
        # 3. MULTI-OBJECTIVE REBALANCING (Hill Climbing)
        # ====================================================================
        if cv_strategy == 'groupby_Sampling_Date':
            df = self._optimize_fold_balance(df, n_folds)
    
        # 4. Final Logging and Saving
        print("\nFinal Fold Distribution:")
        for i in range(n_folds):
            val_df = df[df['fold'] == i]
            trn_df = df[df['fold'] != i]
            
            # Calculate stats for display
            n_samples = len(val_df)
            n_rye = len(val_df[val_df['Species'] == 'Ryegrass_Clover'])
            ratio_rye = n_rye / n_samples if n_samples > 0 else 0
            
            val_months = sorted(list(set(int(x) for x in val_df.Sampling_Date_Month.unique())))
            
            print(f'Fold {i}: Total({n_samples}) | Rye({n_rye}, {ratio_rye:.2%}) | Months: {val_months}')
    
        # Validate global counts
        print("\nGlobal Counts per fold:")
        print(df['fold'].value_counts().sort_index())
        
        df.to_csv('csiro-biomass/trainfold.csv', index=False)
        self.df_wide = df
        return df
    
    def _optimize_fold_balance(self, df: pd.DataFrame, n_folds: int) -> pd.DataFrame:
        """
        Iteratively moves groups (Dates) between folds to minimize variance 
        in Fold Size AND Species Count.
        """
        print("\n--- Starting Hill-Climbing Optimization to balance Size & Species ---")
        
        # Get unique groups and their current assignments
        # We work with a lightweight dictionary for speed
        groups = df.groupby('Sampling_Date').first().reset_index()[['Sampling_Date', 'fold']]
        group_map = dict(zip(groups['Sampling_Date'], groups['fold']))
        
        # Precompute stats for every group to avoid recalculating during loop
        # We need: Size of group, Count of Ryegrass in group
        group_stats = df.groupby('Sampling_Date').apply(
            lambda x: pd.Series({
                'size': len(x),
                'rye': (x['Species'] == 'Ryegrass_Clover').sum()
            })
        ).to_dict('index')
    
        # Current State stats
        fold_stats = {k: {'size': 0, 'rye': 0} for k in range(n_folds)}
        for g, f in group_map.items():
            fold_stats[f]['size'] += group_stats[g]['size']
            fold_stats[f]['rye'] += group_stats[g]['rye']
    
        def get_loss(stats):
            """Calculate loss: StdDev(Sizes) + Weight * StdDev(RyeCounts)"""
            sizes = [v['size'] for v in stats.values()]
            ryes = [v['rye'] for v in stats.values()]
            # Weight 2.0 gives species balance slightly higher priority than raw size
            return np.std(sizes) + 2.0 * np.std(ryes)
    
        current_loss = get_loss(fold_stats)
        print(f"Initial Loss: {current_loss:.4f}")
    
        # Optimization Loop
        # We try random moves. If a move reduces loss, we keep it.
        np.random.seed(self.config.random_state)
        unique_groups = list(group_map.keys())
        
        # Number of iterations: more is better, but diminishing returns.
        # 2000 is usually enough for <50 groups.
        pbar = range(2000) 
        
        for _ in pbar:
            # Pick a random group
            g = np.random.choice(unique_groups)
            current_fold = group_map[g]
            
            # Pick a random target fold (different from current)
            target_fold = np.random.choice([x for x in range(n_folds) if x != current_fold])
            
            # --- Simulate Move ---
            # 1. Remove from current
            fold_stats[current_fold]['size'] -= group_stats[g]['size']
            fold_stats[current_fold]['rye'] -= group_stats[g]['rye']
            
            # 2. Add to target
            fold_stats[target_fold]['size'] += group_stats[g]['size']
            fold_stats[target_fold]['rye'] += group_stats[g]['rye']
            
            new_loss = get_loss(fold_stats)
            
            if new_loss < current_loss:
                # ACCEPT MOVE
                current_loss = new_loss
                group_map[g] = target_fold
            else:
                # REVERT MOVE (Undo changes)
                fold_stats[target_fold]['size'] -= group_stats[g]['size']
                fold_stats[target_fold]['rye'] -= group_stats[g]['rye']
                
                fold_stats[current_fold]['size'] += group_stats[g]['size']
                fold_stats[current_fold]['rye'] += group_stats[g]['rye']
    
        print(f"Final Loss: {current_loss:.4f}")
        
        # Apply changes back to DataFrame
        for g, f in group_map.items():
            df.loc[df['Sampling_Date'] == g, 'fold'] = f
            
        return df


# ============================================================================
# Data Augmentation
# ============================================================================

class AugmentationFactory:
    """
    Factory class for generating Albumentations pipelines.
    
    Provides different pipelines for training and validation.
    """
    
    def __init__(self, img_size: int):
        """
        Args:
            img_size: Image size after resizing
        """
        self.img_size = img_size
    
    def get_train_transforms(self) -> A.Compose:
        """
        Augmentation pipeline for training.
        
        Returns:
            Albumentations Compose object
            
        Why not: Not adding stronger augmentations
            → Balancing overfitting risk and training time
        """
        return A.Compose([
            # Geometric transforms
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ColorJitter(
                brightness=0.2,
                contrast=0.2,
                saturation=0.2,
                hue=0.1,
                p=0.75
            ),
            A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
            A.Affine(scale=(0.9, 1.1),translate_percent=(0.05, 0.05),rotate=(-10, 10),p=0.3),
            A.RandomBrightnessContrast(brightness_limit=0.2,contrast_limit=0.2,p=0.3),
            A.CoarseDropout(num_holes_range=[1, 4],hole_height_range=[0.1, 0.2],hole_width_range=[0.1, 0.2], fill=0, p=0.3),
            # A.RandomShadow(p=0.35),
            # A.RandomFog(p=0.25),
            A.CLAHE(p=0.2),
            A.MotionBlur(p=0.1),
            A.OneOf([
                A.GaussianBlur(blur_limit=(1, 3)),
                A.GaussNoise(std_range=[0.05, 0.1],mean_range=[0, 0],per_channel=True,noise_scale_factor=1),
            ], p=0.3),
            A.Resize(self.img_size, self.img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    
    def get_valid_transforms(self) -> A.Compose:
        """
        Pipeline for validation (no augmentation).
        
        Returns:
            Albumentations Compose object
        """
        return A.Compose([
            A.Resize(self.img_size, self.img_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])


In [5]:
# ============================================================================
# Dataset
# ============================================================================

class BiomassDataset(Dataset):
    """
    Custom Dataset for two-stream architecture.
    
    Splits 2000x1000 images into left and right halves,
    applying independent augmentation to each.
    
    Returns:
        tuple: (img_left, img_right, train_targets, all_targets)
            - img_left: Left image tensor [C, H, W]
            - img_right: Right image tensor [C, H, W]
            - train_targets: Training targets [3]
            - all_targets: Evaluation targets [5]
    """
    
    def __init__(
        self,
        df: pd.DataFrame,
        transforms_fn: Callable[[], A.Compose],
        image_dir: Path,
        train_target_cols: list[str],
        all_target_cols: list[str],
        do_swap: bool = False
    ):
        """
        Args:
            df: DataFrame containing image paths and targets
            transforms_fn: Function returning augmentation pipeline
            image_dir: Image directory path
            train_target_cols: Training target column names
            all_target_cols: Evaluation target column names
        """
        self.df = df
        self.transforms_fn = transforms_fn
        self.image_dir = image_dir
        
        # Convert to numpy arrays for fast access
        self.image_paths = df['image_path'].values
        self.train_targets = df[train_target_cols].values
        self.all_targets = df[all_target_cols].values
        self.do_swap = do_swap
    
    def __len__(self) -> int:
        return len(self.df)
    
    def __getitem__(
        self, 
        idx: int
    ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Get one sample.
        
        Args:
            idx: Sample index
            
        Returns:
            (left_image, right_image, train_targets, eval_targets)
            
        Why not: Not pre-splitting and saving images
            → Considering trade-off between storage and I/O time
        """
        # Get image path and targets
        img_path = self.image_paths[idx]
        train_target = self.train_targets[idx]
        all_target = self.all_targets[idx]
        
        # Load image
        full_path = self.image_dir / Path(img_path).name
        image = cv2.imread(str(full_path))
        
        if image is None:
            raise FileNotFoundError(f"Failed to load image: {full_path}")
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Split into left and right (2000x1000 → 2 x 1000x1000)
        height, width = image.shape[:2]
        mid_point = width // 2
        img_left = image[:, :mid_point]
        img_right = image[:, mid_point:]
        
        # Apply augmentation independently
        # Why not: Not reusing the same transform
        #   → Applying different augmentations to left/right improves data diversity
        transform_left = self.transforms_fn()
        transform_right = self.transforms_fn()
        
        img_left_tensor = transform_left(image=img_left)['image']
        img_right_tensor = transform_right(image=img_right)['image']

        if self.do_swap and (torch.rand(1) > 0.5):
            img_left_tensor, img_right_tensor = img_right_tensor, img_left_tensor
        
        # Convert targets to tensors
        train_target_tensor = torch.tensor(train_target, dtype=torch.float32)
        all_target_tensor = torch.tensor(all_target, dtype=torch.float32)
        
        return img_left_tensor, img_right_tensor, train_target_tensor, all_target_tensor

In [6]:
# ==================== MODEL COMPONENTS ====================

def _build_dino_by_name(backbone_name: str, pretrained: bool = False):
    model = timm.create_model(backbone_name, pretrained=pretrained, num_classes=0)
    feat_dim = model.embed_dim
    if hasattr(model, "patch_embed"):
        input_res = model.patch_embed.img_size
        if isinstance(input_res, (tuple, list)):
            input_res = input_res[0]
    else:
        input_res = 518
    return model, feat_dim, input_res


def _make_edges(length: int, n_parts: int) -> List[Tuple[int, int]]:
    step = length // n_parts
    edges = [(i * step, (i + 1) * step) for i in range(n_parts)]
    edges[-1] = (edges[-1][0], length)
    return edges


class FiLM(nn.Module):
    def __init__(self, in_dim: int) -> None:
        super().__init__()
        hidden = max(64, in_dim // 2)
        self.mlp = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Linear(hidden, in_dim * 2),
        )
    
    def forward(self, context: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        gb = self.mlp(context)
        gamma, beta = torch.chunk(gb, 2, dim=1)
        return gamma, beta


class AttentionPooling(nn.Module):
    def __init__(self, dim: int) -> None:
        super().__init__()
        self.query = nn.Linear(dim, dim)
        self.key = nn.Linear(dim, dim)
        self.scale = dim ** -0.5
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        q = self.query(x.mean(dim=1, keepdim=True))
        k = self.key(x)
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = F.softmax(attn, dim=-1)
        return (attn @ x).squeeze(1)


# ==================== RATIO MODELS ====================

class SoftmaxRatioDINO(nn.Module):
    """Softmax Ratio Model: Predict Total + component ratios."""
    
    def __init__(
        self,
        backbone_name: str = "vit_base_patch14_reg4_dinov2.lvd142m",
        grid: Tuple[int, int] = (2, 2),
        pretrained: bool = False,
        dropout: float = 0.2,
        hidden_ratio: float = 0.5,
        use_film: bool = True,
        use_attention_pool: bool = True,
        ratio_temperature: float = 1.0,
    ) -> None:
        super().__init__()
        
        self.backbone, feat_dim, input_res = _build_dino_by_name(backbone_name, pretrained)
        self.input_res = int(input_res)
        self.feat_dim = feat_dim
        self.grid = tuple(grid)
        self.use_film = use_film
        self.use_attention_pool = use_attention_pool
        self.ratio_temperature = ratio_temperature
        
        if use_film:
            self.film_left = FiLM(feat_dim)
            self.film_right = FiLM(feat_dim)
        
        if use_attention_pool:
            self.attn_pool_left = AttentionPooling(feat_dim)
            self.attn_pool_right = AttentionPooling(feat_dim)
        
        self.combined_dim = feat_dim * 2
        hidden_dim = max(64, int(self.combined_dim * hidden_ratio))
        
        self.shared_proj = nn.Sequential(
            nn.LayerNorm(self.combined_dim),
            nn.Linear(self.combined_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
        )
        
        def _make_head(in_dim: int, out_dim: int = 1) -> nn.Sequential:
            return nn.Sequential(
                nn.Linear(in_dim, in_dim),
                nn.GELU(),
                nn.Dropout(dropout * 0.5),
                nn.Linear(in_dim, out_dim),
            )
        
        self.head_total = _make_head(hidden_dim, 1)
        self.head_ratios = _make_head(hidden_dim, 3)
        self.softplus = nn.Softplus(beta=1.0)
    
    def _collect_tiles(self, x: torch.Tensor) -> List[torch.Tensor]:
        _, C, H, W = x.shape
        r, c = self.grid
        rows = _make_edges(H, r)
        cols = _make_edges(W, c)
        tiles = []
        for (y0, y1) in rows:
            for (x0, x1) in cols:
                tile = x[:, :, y0:y1, x0:x1]
                tile = F.interpolate(tile, size=(self.input_res, self.input_res), mode="bilinear", align_corners=False)
                tiles.append(tile)
        return tiles
    
    def _extract_tiles_fused(self, x_left: torch.Tensor, x_right: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        B = x_left.size(0)
        tiles_left = self._collect_tiles(x_left)
        tiles_right = self._collect_tiles(x_right)
        num_tiles = len(tiles_left)
        
        all_tiles = torch.cat(tiles_left + tiles_right, dim=0)
        all_feats = self.backbone(all_tiles)
        
        total_tiles = 2 * num_tiles
        all_feats = all_feats.view(total_tiles, B, -1).permute(1, 0, 2)
        feats_left = all_feats[:, :num_tiles, :]
        feats_right = all_feats[:, num_tiles:, :]
        return feats_left, feats_right
    
    def forward(self, x_left: torch.Tensor, x_right: torch.Tensor) -> Tuple[torch.Tensor, ...]:
        tiles_left, tiles_right = self._extract_tiles_fused(x_left, x_right)
        
        ctx_left = tiles_left.mean(dim=1)
        ctx_right = tiles_right.mean(dim=1)
        
        if self.use_film:
            gamma_l, beta_l = self.film_left(ctx_right)
            gamma_r, beta_r = self.film_right(ctx_left)
            tiles_left = tiles_left * (1 + gamma_l.unsqueeze(1)) + beta_l.unsqueeze(1)
            tiles_right = tiles_right * (1 + gamma_r.unsqueeze(1)) + beta_r.unsqueeze(1)
        
        if self.use_attention_pool:
            f_l = self.attn_pool_left(tiles_left)
            f_r = self.attn_pool_right(tiles_right)
        else:
            f_l = tiles_left.mean(dim=1)
            f_r = tiles_right.mean(dim=1)
        
        f = torch.cat([f_l, f_r], dim=1)
        f = self.shared_proj(f)
        
        total = self.softplus(self.head_total(f))
        logits = self.head_ratios(f)
        ratios = F.softmax(logits / self.ratio_temperature, dim=1)
        green_ratio, dead_ratio, clover_ratio = ratios[:, 0:1], ratios[:, 1:2], ratios[:, 2:3]
        
        green = total * green_ratio
        dead = total * dead_ratio
        clover = total * clover_ratio
        gdm = green + clover
        
        return green, dead, clover, gdm, total


class HierarchicalRatioDINO(nn.Module):
    """Hierarchical Ratio Model: Total → GDM/Total → Green/GDM."""
    
    def __init__(
        self,
        backbone_name: str = "vit_base_patch14_reg4_dinov2.lvd142m",
        grid: Tuple[int, int] = (2, 2),
        pretrained: bool = False,
        dropout: float = 0.2,
        hidden_ratio: float = 0.5,
        use_film: bool = True,
        use_attention_pool: bool = True,
    ) -> None:
        super().__init__()
        
        self.backbone, feat_dim, input_res = _build_dino_by_name(backbone_name, pretrained)
        self.input_res = int(input_res)
        self.feat_dim = feat_dim
        self.grid = tuple(grid)
        self.use_film = use_film
        self.use_attention_pool = use_attention_pool
        
        if use_film:
            self.film_left = FiLM(feat_dim)
            self.film_right = FiLM(feat_dim)
        
        if use_attention_pool:
            self.attn_pool_left = AttentionPooling(feat_dim)
            self.attn_pool_right = AttentionPooling(feat_dim)
        
        self.combined_dim = feat_dim * 2
        hidden_dim = max(64, int(self.combined_dim * hidden_ratio))
        
        self.shared_proj = nn.Sequential(
            nn.LayerNorm(self.combined_dim),
            nn.Linear(self.combined_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
        )
        
        def _make_head(in_dim: int, out_dim: int = 1) -> nn.Sequential:
            return nn.Sequential(
                nn.Linear(in_dim, in_dim),
                nn.GELU(),
                nn.Dropout(dropout * 0.5),
                nn.Linear(in_dim, out_dim),
            )
        
        self.head_total = _make_head(hidden_dim, 1)
        self.head_alive_ratio = _make_head(hidden_dim, 1)
        self.head_green_ratio = _make_head(hidden_dim, 1)
        self.softplus = nn.Softplus(beta=1.0)
    
    def _collect_tiles(self, x: torch.Tensor) -> List[torch.Tensor]:
        _, C, H, W = x.shape
        r, c = self.grid
        rows = _make_edges(H, r)
        cols = _make_edges(W, c)
        tiles = []
        for (y0, y1) in rows:
            for (x0, x1) in cols:
                tile = x[:, :, y0:y1, x0:x1]
                tile = F.interpolate(tile, size=(self.input_res, self.input_res), mode="bilinear", align_corners=False)
                tiles.append(tile)
        return tiles
    
    def _extract_tiles_fused(self, x_left: torch.Tensor, x_right: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        B = x_left.size(0)
        tiles_left = self._collect_tiles(x_left)
        tiles_right = self._collect_tiles(x_right)
        num_tiles = len(tiles_left)
        
        all_tiles = torch.cat(tiles_left + tiles_right, dim=0)
        all_feats = self.backbone(all_tiles)
        
        total_tiles = 2 * num_tiles
        all_feats = all_feats.view(total_tiles, B, -1).permute(1, 0, 2)
        feats_left = all_feats[:, :num_tiles, :]
        feats_right = all_feats[:, num_tiles:, :]
        return feats_left, feats_right
    
    def forward(self, x_left: torch.Tensor, x_right: torch.Tensor) -> Tuple[torch.Tensor, ...]:
        tiles_left, tiles_right = self._extract_tiles_fused(x_left, x_right)
        
        ctx_left = tiles_left.mean(dim=1)
        ctx_right = tiles_right.mean(dim=1)
        
        if self.use_film:
            gamma_l, beta_l = self.film_left(ctx_right)
            gamma_r, beta_r = self.film_right(ctx_left)
            tiles_left = tiles_left * (1 + gamma_l.unsqueeze(1)) + beta_l.unsqueeze(1)
            tiles_right = tiles_right * (1 + gamma_r.unsqueeze(1)) + beta_r.unsqueeze(1)
        
        if self.use_attention_pool:
            f_l = self.attn_pool_left(tiles_left)
            f_r = self.attn_pool_right(tiles_right)
        else:
            f_l = tiles_left.mean(dim=1)
            f_r = tiles_right.mean(dim=1)
        
        f = torch.cat([f_l, f_r], dim=1)
        f = self.shared_proj(f)
        
        total = self.softplus(self.head_total(f))
        alive_ratio = torch.sigmoid(self.head_alive_ratio(f))
        green_ratio = torch.sigmoid(self.head_green_ratio(f))
        
        gdm = total * alive_ratio
        dead = total - gdm
        green = gdm * green_ratio
        clover = gdm - green
        
        dead = F.relu(dead)
        clover = F.relu(clover)
        
        return green, dead, clover, gdm, total


class DirectDINO(nn.Module):
    """Direct Model: Predict Total, Green, GDM directly; derive Dead, Clover."""
    
    def __init__(
        self,
        backbone_name: str = "vit_base_patch14_reg4_dinov2.lvd142m",
        grid: Tuple[int, int] = (2, 2),
        pretrained: bool = False,
        dropout: float = 0.2,
        hidden_ratio: float = 0.5,
        use_film: bool = True,
        use_attention_pool: bool = True,
    ) -> None:
        super().__init__()
        
        self.backbone, feat_dim, input_res = _build_dino_by_name(backbone_name, pretrained)
        self.input_res = int(input_res)
        self.feat_dim = feat_dim
        self.grid = tuple(grid)
        self.use_film = use_film
        self.use_attention_pool = use_attention_pool
        
        if use_film:
            self.film_left = FiLM(feat_dim)
            self.film_right = FiLM(feat_dim)
        
        if use_attention_pool:
            self.attn_pool_left = AttentionPooling(feat_dim)
            self.attn_pool_right = AttentionPooling(feat_dim)
        
        self.combined_dim = feat_dim * 2
        hidden_dim = max(64, int(self.combined_dim * hidden_ratio))
        
        self.shared_proj = nn.Sequential(
            nn.LayerNorm(self.combined_dim),
            nn.Linear(self.combined_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
        )
        
        def _make_head(in_dim: int, out_dim: int = 1) -> nn.Sequential:
            return nn.Sequential(
                nn.Linear(in_dim, in_dim),
                nn.GELU(),
                nn.Dropout(dropout * 0.5),
                nn.Linear(in_dim, out_dim),
            )
        
        self.head_total = _make_head(hidden_dim, 1)
        self.head_green = _make_head(hidden_dim, 1)
        self.head_gdm = _make_head(hidden_dim, 1)
        self.softplus = nn.Softplus(beta=1.0)
    
    def _collect_tiles(self, x: torch.Tensor) -> List[torch.Tensor]:
        _, C, H, W = x.shape
        r, c = self.grid
        rows = _make_edges(H, r)
        cols = _make_edges(W, c)
        tiles = []
        for (y0, y1) in rows:
            for (x0, x1) in cols:
                tile = x[:, :, y0:y1, x0:x1]
                tile = F.interpolate(tile, size=(self.input_res, self.input_res), mode="bilinear", align_corners=False)
                tiles.append(tile)
        return tiles
    
    def _extract_tiles_fused(self, x_left: torch.Tensor, x_right: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        B = x_left.size(0)
        tiles_left = self._collect_tiles(x_left)
        tiles_right = self._collect_tiles(x_right)
        num_tiles = len(tiles_left)
        
        all_tiles = torch.cat(tiles_left + tiles_right, dim=0)
        all_feats = self.backbone(all_tiles)
        
        total_tiles = 2 * num_tiles
        all_feats = all_feats.view(total_tiles, B, -1).permute(1, 0, 2)
        feats_left = all_feats[:, :num_tiles, :]
        feats_right = all_feats[:, num_tiles:, :]
        return feats_left, feats_right
    
    def forward(self, x_left: torch.Tensor, x_right: torch.Tensor) -> Tuple[torch.Tensor, ...]:
        tiles_left, tiles_right = self._extract_tiles_fused(x_left, x_right)
        
        ctx_left = tiles_left.mean(dim=1)
        ctx_right = tiles_right.mean(dim=1)
        
        if self.use_film:
            gamma_l, beta_l = self.film_left(ctx_right)
            gamma_r, beta_r = self.film_right(ctx_left)
            tiles_left = tiles_left * (1 + gamma_l.unsqueeze(1)) + beta_l.unsqueeze(1)
            tiles_right = tiles_right * (1 + gamma_r.unsqueeze(1)) + beta_r.unsqueeze(1)
        
        if self.use_attention_pool:
            f_l = self.attn_pool_left(tiles_left)
            f_r = self.attn_pool_right(tiles_right)
        else:
            f_l = tiles_left.mean(dim=1)
            f_r = tiles_right.mean(dim=1)
        
        f = torch.cat([f_l, f_r], dim=1)
        f = self.shared_proj(f)
        
        # Predict Total, Green, GDM directly (all positive via softplus)
        total_raw = self.softplus(self.head_total(f))
        green_raw = self.softplus(self.head_green(f))
        gdm_raw = self.softplus(self.head_gdm(f))
        
        # Enforce constraints: Total >= GDM >= Green >= 0
        total = total_raw
        gdm = torch.minimum(gdm_raw, total)
        green = torch.minimum(green_raw, gdm)
        
        # Derive Dead and Clover
        dead = total - gdm
        clover = gdm - green
        
        # Ensure non-negative (numerical safety)
        dead = F.relu(dead)
        clover = F.relu(clover)
        
        return green, dead, clover, gdm, total

In [7]:
# ============================================================================
# Loss Function
# ============================================================================

class WeightedBiomassLoss(nn.Module):
    """
    Weighted loss for three targets.
    
    Weight losses according to importance of each target,
    using SmoothL1Loss for robust learning against outliers.
    """
    
    def __init__(self, loss_weights: dict[str, float]):
        """
        Args:
            loss_weights: Weights for each target
        """
        super().__init__()
        self.criterion = nn.SmoothL1Loss()  # A variant of Huber loss
        self.weights = loss_weights
    
    def forward(
        self,
        predictions: tuple[torch.Tensor, torch.Tensor, torch.Tensor],
        targets: torch.Tensor
    ) -> torch.Tensor:
        """
        Calculate weighted loss.
        
        Args:
            predictions: (total, gdm, green) each [B, 1]
            targets: [B, 3] (in order: total, gdm, green)
            
        Returns:
            Weighted total loss
        """
        pred_total, pred_gdm, pred_green = predictions
        
        # Decompose targets
        true_total = targets[:, 0:1]  # Maintain [B, 1] shape
        true_gdm = targets[:, 1:2]
        true_green = targets[:, 2:3]
        
        # Calculate each loss
        loss_total = self.criterion(pred_total, true_total)
        loss_gdm = self.criterion(pred_gdm, true_gdm)
        loss_green = self.criterion(pred_green, true_green)
        
        # Weighted sum
        total_loss = (
            self.weights['total_loss'] * loss_total +
            self.weights['gdm_loss'] * loss_gdm +
            self.weights['green_loss'] * loss_green
        )
        
        return total_loss


# ============================================================================
# Evaluation Metrics
# ============================================================================

class CompetitionScorer:
    """
    Class for calculating competition evaluation metric (weighted R²)
    and individual R² per column.
    """
    
    def __init__(self, r2_weights: list[float]):
        self.r2_weights = np.array(r2_weights)
        self.target_names = ['Dry_Green', 'Dry_Dead', 'Dry_Clover', 'GDM', 'Total']
    
    def _reconstruct_predictions(self, preds_dict: dict[str, np.ndarray]) -> np.ndarray:
        """Helper to reconstruct the 5-column prediction matrix."""
        pred_total = preds_dict['total']
        pred_gdm = preds_dict['gdm']
        pred_green = preds_dict['green']
        pred_clover = preds_dict['clover']
        pred_dead = preds_dict['dead']
        # # Estimate remaining two (clip negative values)
        # pred_clover = np.maximum(0, pred_gdm - pred_green)
        # pred_dead = np.maximum(0, pred_total - pred_gdm)
        
        # Combine five predictions (Order: green, dead, clover, gdm, total)
        return np.stack([
            pred_green, pred_dead, pred_clover, pred_gdm, pred_total
        ], axis=1)

    def calculate_score(self, preds_dict: dict[str, np.ndarray], targets_5: np.ndarray) -> float:
        """Calculate weighted R² score (Competition Metric)."""
        y_preds = self._reconstruct_predictions(preds_dict)
        r2_scores = r2_score(targets_5, y_preds, multioutput='raw_values')
        return float(np.sum(r2_scores * self.r2_weights))
    
    def get_individual_r2(self, preds_dict: dict[str, np.ndarray], targets_5: np.ndarray) -> dict[str, float]:
        """Returns dictionary of R² scores for each target."""
        y_preds = self._reconstruct_predictions(preds_dict)
        r2_scores = r2_score(targets_5, y_preds, multioutput='raw_values')
        
        return {name: float(score) for name, score in zip(self.target_names, r2_scores)}

    def print_r2_report(self, preds_dict: dict[str, np.ndarray], targets_5: np.ndarray):
        """Prints a clean table of R² scores."""
        scores = self.get_individual_r2(preds_dict, targets_5)
        weighted = self.calculate_score(preds_dict, targets_5)
        
        print(f"\n{'Target':<15} | {'R² Score':<10}")
        print("-" * 28)
        for name, score in scores.items():
            print(f"{name:<15} | {score:.4f}")
        print("-" * 28)
        print(f"{'Weighted Avg':<15} | {weighted:.4f}\n")

    def calculate_individual_scores(
            self,
            preds_dict: dict[str, np.ndarray],
            targets_5: np.ndarray
        ) -> dict[str, float]:
        """
        Calculate individual R² scores for each target.
        
        Args:
            preds_dict: {'total': [N], 'gdm': [N], 'green': [N]}
            targets_5: [N, 5] (in order: green, dead, clover, gdm, total)
            
        Returns:
            Dictionary of R² scores for each target
        """
        # Get predictions
        pred_total = preds_dict['total']
        pred_gdm = preds_dict['gdm']
        pred_green = preds_dict['green']
        pred_clover = preds_dict['clover']
        pred_dead = preds_dict['dead']
        # Estimate remaining two (clip negative values)
        # pred_clover = np.maximum(0, pred_gdm - pred_green)
        # pred_dead = np.maximum(0, pred_total - pred_gdm)
        
        # Combine five predictions (green, dead, clover, gdm, total)
        y_preds = np.stack([
            pred_green, pred_dead, pred_clover, pred_gdm, pred_total
        ], axis=1)
        
        # Calculate R² for each target
        r2_scores = r2_score(targets_5, y_preds, multioutput='raw_values')
        
        target_names = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
        
        return {name: float(score) for name, score in zip(target_names, r2_scores)}


# ============================================================================
# Training History Tracker
# ============================================================================

@dataclass
class TrainingHistory:
    """
    Data class for tracking training history.
    
    Stores losses, scores, and predictions for visualization.
    """
    train_losses: list[float] = field(default_factory=list)
    valid_losses: list[float] = field(default_factory=list)
    valid_scores: list[float] = field(default_factory=list)
    
    # For final validation predictions
    final_preds: Optional[dict[str, np.ndarray]] = None
    final_targets: Optional[np.ndarray] = None
    
    def add_epoch(
        self,
        train_loss: float,
        valid_loss: float,
        valid_score: float
    ) -> None:
        """Add one epoch's metrics."""
        self.train_losses.append(train_loss)
        self.valid_losses.append(valid_loss)
        self.valid_scores.append(valid_score)
    
    def set_final_predictions(
        self,
        preds: dict[str, np.ndarray],
        targets: np.ndarray
    ) -> None:
        """Store final predictions for scatter plots."""
        self.final_preds = preds
        self.final_targets = targets


# ============================================================================
# Training and Validation Loops
# ============================================================================

class Trainer:
    """
    Class for managing training and validation loops with history tracking.
    """
    
    def __init__(
        self,
        model: nn.Module,
        criterion: nn.Module,
        device: torch.device,
        scorer: CompetitionScorer
    ):
        """
        Args:
            model: Model to train
            criterion: Loss function
            device: Device (CPU/GPU)
            scorer: Evaluation metric calculator
        """
        self.model = model
        self.criterion = criterion
        self.device = device
        self.scorer = scorer
        self.history = TrainingHistory()
    
    def train_one_epoch(
        self,
        loader: DataLoader,
        optimizer: optim.Optimizer
    ) -> float:
        """
        Train for one epoch.
        
        Args:
            loader: Training DataLoader
            optimizer: Optimizer
            
        Returns:
            Average loss for the epoch
        """
        self.model.train()
        epoch_loss = 0.0
        
        pbar = tqdm(loader, desc="Training", leave=False)
        
        for img_left, img_right, train_targets, _ in pbar:
            # Transfer to device
            img_left = img_left.to(self.device)
            img_right = img_right.to(self.device)
            targets = train_targets.to(self.device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            pred_green, _, _, pred_gdm, pred_total = self.model(img_left, img_right)
                
            # Calculate loss
            predictions = (pred_total, pred_gdm, pred_green)
            
            # Calculate loss
            loss = self.criterion(predictions, targets)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Record
            epoch_loss += loss.item()
            pbar.set_postfix(loss=f'{loss.item():.4f}')
        
        return epoch_loss / len(loader)
    
    def validate_one_epoch(
        self,
        loader: DataLoader,
        store_predictions: bool = False
    ) -> tuple[float, float]:
        """
        Validate for one epoch.
        
        Args:
            loader: Validation DataLoader
            store_predictions: Whether to store predictions for visualization
            
        Returns:
            (average loss for epoch, R² score)
        """
        self.model.eval()
        epoch_loss = 0.0
        
        # Collect predictions and targets
        all_preds = {'total': [], 'gdm': [], 'green': [], 'dead': [], 'clover':[]}
        all_targets = []
        
        with torch.no_grad():
            pbar = tqdm(loader, desc="Validating", leave=False)
            
            for img_left, img_right, train_targets, all_targets_5 in pbar:
                img_left = img_left.to(self.device)
                img_right = img_right.to(self.device)
                train_targets = train_targets.to(self.device)
                
                # Forward pass
                pred_green, pred_dead, pred_clover, pred_gdm, pred_total = self.model(img_left, img_right)
                
                # Calculate loss
                predictions = (pred_total, pred_gdm, pred_green)
                loss = self.criterion(predictions, train_targets)
                epoch_loss += loss.item()
                
                # Collect predictions (convert to numpy on CPU)
                all_preds['total'].append(pred_total.cpu().numpy())
                all_preds['gdm'].append(pred_gdm.cpu().numpy())
                all_preds['green'].append(pred_green.cpu().numpy())
                all_preds['dead'].append(pred_dead.cpu().numpy())
                all_preds['clover'].append(pred_clover.cpu().numpy())
                all_targets.append(all_targets_5.cpu().numpy())
        
        # Concatenate batches
        preds_np = {
            k: np.concatenate(v).flatten() 
            for k, v in all_preds.items()
        }
        targets_np = np.concatenate(all_targets)
        
        # Store predictions if requested
        if store_predictions:
            self.history.set_final_predictions(preds_np, targets_np)
        
        # Calculate R² score
        score = self.scorer.calculate_score(preds_np, targets_np)
        self.scorer.print_r2_report(preds_np, targets_np)
        
        avg_loss = epoch_loss / len(loader)
        return avg_loss, score


# ============================================================================
# Visualization
# ============================================================================

class TrainingVisualizer:
    """
    Class for creating comprehensive training visualizations.
    
    Generates and saves:
    1. Learning curves (loss and R² over epochs)
    2. Fold comparison (bar chart of scores across folds)
    3. Prediction scatter plots (predicted vs. actual for all targets)
    """
    
    def __init__(self, output_dir: Path, target_names: list[str]):
        """
        Args:
            output_dir: Directory to save plots
            target_names: List of target names for labeling
        """
        self.output_dir = output_dir
        self.target_names = target_names
        
        # Set style
        sns.set_style("whitegrid")
        plt.rcParams['figure.dpi'] = 100
        plt.rcParams['savefig.dpi'] = 300
        plt.rcParams['font.size'] = 10
    
    def plot_learning_curves(
        self,
        history: TrainingHistory,
        fold: int,
        freeze_epoch: int
    ) -> None:
        """
        Plot learning curves showing loss and R² score over epochs.
        
        Args:
            history: Training history object
            fold: Fold number
            freeze_epoch: Epoch where backbone was unfrozen
        """
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        epochs = range(1, len(history.train_losses) + 1)
        
        # Plot 1: Loss curves
        axes[0].plot(epochs, history.train_losses, label='Train Loss', marker='o', linewidth=2)
        axes[0].plot(epochs, history.valid_losses, label='Valid Loss', marker='s', linewidth=2)
        axes[0].axvline(x=freeze_epoch, color='red', linestyle='--', 
                        label=f'Unfreeze (Epoch {freeze_epoch})', alpha=0.7)
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].set_title(f'Fold {fold}: Training and Validation Loss')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Plot 2: R² score curve
        axes[1].plot(epochs, history.valid_scores, label='Valid R² Score', 
                     marker='D', color='green', linewidth=2)
        axes[1].axvline(x=freeze_epoch, color='red', linestyle='--', 
                        label=f'Unfreeze (Epoch {freeze_epoch})', alpha=0.7)
        axes[1].axhline(y=max(history.valid_scores), color='orange', linestyle=':', 
                        label=f'Best R² = {max(history.valid_scores):.4f}', alpha=0.7)
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('R² Score')
        axes[1].set_title(f'Fold {fold}: Validation R² Score')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        save_path = self.output_dir / f'fold{fold}_learning_curves.png'
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()
        
        print(f"  📊 Saved learning curves: {save_path}")
    
    def plot_prediction_scatter(
        self,
        history: TrainingHistory,
        fold: int,
        scorer: CompetitionScorer
    ) -> None:
        """
        Plot scatter plots of predicted vs. actual values for all targets.
        
        Args:
            history: Training history with final predictions
            fold: Fold number
            scorer: Scorer to calculate individual R² scores
        """
        if history.final_preds is None or history.final_targets is None:
            print(f"  ⚠️  No predictions stored for fold {fold}")
            return
        
        # Reconstruct all 5 predictions
        pred_total = history.final_preds['total']
        pred_gdm = history.final_preds['gdm']
        pred_green = history.final_preds['green']
        pred_clover = np.maximum(0, pred_gdm - pred_green)
        pred_dead = np.maximum(0, pred_total - pred_gdm)
        
        predictions = [pred_green, pred_dead, pred_clover, pred_gdm, pred_total]
        targets = history.final_targets
        
        # Calculate individual R² scores
        individual_r2 = scorer.calculate_individual_scores(
            history.final_preds,
            history.final_targets
        )
        
        # Create subplots
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        for idx, (pred, target_name) in enumerate(zip(predictions, self.target_names)):
            ax = axes[idx]
            true_vals = targets[:, idx]
            
            # Scatter plot
            ax.scatter(true_vals, pred, alpha=0.5, s=20)
            
            # Perfect prediction line
            min_val = min(true_vals.min(), pred.min())
            max_val = max(true_vals.max(), pred.max())
            ax.plot([min_val, max_val], [min_val, max_val], 
                   'r--', linewidth=2, label='Perfect Prediction')
            
            # Labels and title
            ax.set_xlabel('Actual Value (g)')
            ax.set_ylabel('Predicted Value (g)')
            r2 = individual_r2[target_name]
            ax.set_title(f'{target_name}\nR² = {r2:.4f}')
            ax.legend()
            ax.grid(True, alpha=0.3)
        
        # Remove extra subplot
        axes[5].axis('off')
        
        plt.suptitle(f'Fold {fold}: Prediction vs. Actual Scatter Plots', 
                     fontsize=14, fontweight='bold', y=0.995)
        plt.tight_layout()
        
        save_path = self.output_dir / f'fold{fold}_scatter_plots.png'
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()
        
        print(f"  📊 Saved scatter plots: {save_path}")
    
    def plot_fold_comparison(
        self,
        fold_scores: dict[int, float]
    ) -> None:
        """
        Plot bar chart comparing R² scores across all folds.
        
        Args:
            fold_scores: Dictionary mapping fold number to best R² score
        """
        folds = sorted(fold_scores.keys())
        scores = [fold_scores[f] for f in folds]
        mean_score = np.mean(scores)
        std_score = np.std(scores)
        
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Bar plot
        bars = ax.bar(folds, scores, color='skyblue', edgecolor='navy', linewidth=1.5)
        
        # Add value labels on bars
        for fold, score, bar in zip(folds, scores, bars):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2., height,
                   f'{score:.4f}',
                   ha='center', va='bottom', fontweight='bold')
        
        # Mean line
        ax.axhline(y=mean_score, color='red', linestyle='--', linewidth=2,
                  label=f'Mean R² = {mean_score:.4f} ± {std_score:.4f}')
        
        # Labels and title
        ax.set_xlabel('Fold', fontsize=12, fontweight='bold')
        ax.set_ylabel('Best R² Score', fontsize=12, fontweight='bold')
        ax.set_title('Cross-Validation Performance: R² Score Comparison Across Folds', 
                    fontsize=14, fontweight='bold')
        ax.set_xticks(folds)
        ax.set_xticklabels([f'Fold {f}' for f in folds])
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        save_path = self.output_dir / 'fold_comparison.png'
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()
        
        print(f"\n📊 Saved fold comparison: {save_path}")
        print(f"📈 Overall CV Score: {mean_score:.4f} ± {std_score:.4f}")


class WarmupCosineRestarts(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, warmup_epochs, T_0, T_mult=1, eta_min=0, last_epoch=-1):
        self.warmup_epochs = warmup_epochs
        self.cosine = CosineAnnealingWarmRestarts(
            optimizer, T_0=T_0, T_mult=T_mult, eta_min=eta_min
        )
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [group['lr'] for group in self.optimizer.param_groups]

    def step(self, epoch=None):
        if self.last_epoch < self.warmup_epochs:
            # warmup
            for param_group in self.optimizer.param_groups:
                param_group['lr'] = (
                    param_group['initial_lr'] * (self.last_epoch + 1) / self.warmup_epochs
                )
        else:
            self.cosine.step(epoch - self.warmup_epochs if epoch is not None else None)

        super().step(epoch)


# ============================================================================
# Two-Stage Training Pipeline with Visualization
# ============================================================================

class TwoStageTrainingPipeline:
    """
    Pipeline executing two-stage training: Freeze→Unfreeze with visualization.
    
    Stage 1: Freeze backbone and train heads only
    Stage 2: Fine-tune entire model
    """
    
    def __init__(self, config: Config):
        """
        Args:
            config: Configuration object
        """
        self.config = config
        self.data_prep = DataPreparator(config)
        self.aug_factory = AugmentationFactory(config.img_size)
        self.scorer = CompetitionScorer(config.r2_weights)
        self.visualizer = TrainingVisualizer(config.output_dir, config.all_target_cols)
        
        # Store fold scores for comparison
        self.fold_scores: dict[int, float] = {}
    
    def run_fold(self, fold: int) -> None:
        """
        Execute training for one fold with visualization.
        
        Args:
            fold: Fold number
        """
        print(f"\n{'='*70}")
        print(f"🚀 Starting Fold {fold} Training (Two-Stage)")
        print(f"{'='*70}")
        
        start_time = time.time()
        
        # Prepare data
        df = self.data_prep.df_wide
        train_df = df[df['fold'] != fold].reset_index(drop=True)
        valid_df = df[df['fold'] == fold].reset_index(drop=True)
        
        print(f"Training data: {len(train_df)} images | Validation data: {len(valid_df)} images")
        
        # Create datasets
        train_dataset = BiomassDataset(
            train_df,
            self.aug_factory.get_train_transforms,
            self.config.image_dir,
            self.config.train_target_cols,
            self.config.all_target_cols
        )
        
        valid_dataset = BiomassDataset(
            valid_df,
            self.aug_factory.get_valid_transforms,
            self.config.image_dir,
            self.config.train_target_cols,
            self.config.all_target_cols
        )
        
        # Create DataLoaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=self.config.batch_size,
            shuffle=True,
            num_workers=self.config.num_workers,
            pin_memory=True
        )
        
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=self.config.batch_size * 2,
            shuffle=False,
            num_workers=self.config.num_workers,
            pin_memory=True
        )
        
        # Initialize model
        # model_base = BiomassModel(
        #     self.config.model_name,
        #     self.config.pretrained
        # )
        # variant = self.config.variant
        # if variant == "tiled_film":
        #     model_base = TwoStreamDINOTiledFiLM(self.config.backbone_name, grid=self.config.GRID, dropout=self.config.DROPOUT)
        # elif variant == "tiled":
        #     model_base = TwoStreamDINOTiled(self.config.backbone_name, grid=self.config.GRID, dropout=self.config.DROPOUT)
        # else:
        #     model_base = TwoStreamDINOPlain(self.config.backbone_name, dropout=self.config.DROPOUT)
        model_base = DirectDINO(self.config.backbone_name, grid=self.config.GRID, pretrained= True)
        
        # Multi-GPU support
        if torch.cuda.device_count() > 1:
            print(f"Using {torch.cuda.device_count()} GPUs (DataParallel)")
            model = nn.DataParallel(model_base)
        else:
            model = model_base
        
        model.to(self.config.device)
        
        # Loss function
        criterion = WeightedBiomassLoss(self.config.loss_weights).to(self.config.device)
        
        # Trainer instance
        trainer = Trainer(model, criterion, self.config.device, self.scorer)

        best_score = -float('inf')
        
        # ===== Stage 1: Freeze Backbone =====
        print(f"\n--- Stage 1: Backbone Frozen (Epoch 1-{self.config.freeze_epochs}) ---")
        self._freeze_backbone(model)
        
        optimizer = optim.Adam(
            filter(lambda p: p.requires_grad, model.parameters()),
            lr=self.config.learning_rate
        )
        
        # --- Cosine Annealing for Stage 1 ---
        scheduler = optim.lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=self.config.freeze_epochs,
            eta_min=1e-6
        )
        
        for epoch in range(1, self.config.freeze_epochs + 1):
            print(f"\nEpoch {epoch}/{self.config.epochs} (Stage 1)")
            
            train_loss = trainer.train_one_epoch(train_loader, optimizer)
            valid_loss, score = trainer.validate_one_epoch(valid_loader)
            
            trainer.history.add_epoch(train_loss, valid_loss, score)
            
            scheduler.step()
            
            print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f} | R²: {score:.4f}")
            
            if score > best_score:
                best_score = score
                self._save_model(model, fold)
                print("✨ R² score improved! Saving model")

        # best_model_path = self.config.output_dir / f'best_model_fold{fold}.pth'
        # print(f"\n♻️ Loading best weights from Stage 1 before fine-tuning: {best_model_path}")
        # state_dict = torch.load(best_model_path, weights_only=True)
        # if isinstance(model, nn.DataParallel):
        #     model.module.load_state_dict(state_dict)
        # else:
        #     model.load_state_dict(state_dict)
        
        # ===== Stage 2: Fine-tune Entire Model =====
        print(f"\n--- Stage 2: Full Fine-tuning (Epoch {self.config.freeze_epochs+1}-{self.config.epochs}) ---")
        self._unfreeze_backbone(model)
        
        optimizer = optim.Adam(
            model.parameters(),
            lr=self.config.finetune_lr
        )
        
        scheduler = WarmupCosineRestarts(
            optimizer,
            warmup_epochs=3,
            T_0=5,
            T_mult=2,
            eta_min=1e-8
        )

        patience = 10
        epochs_no_improve = 0
        
        for epoch in range(self.config.freeze_epochs + 1, self.config.epochs + 1):
            print(f"\nEpoch {epoch}/{self.config.epochs} (Stage 2)")
            
            train_loss = trainer.train_one_epoch(train_loader, optimizer)
            
            store_preds = (epoch == self.config.epochs)
            valid_loss, score = trainer.validate_one_epoch(valid_loader, store_predictions=store_preds)
            
            trainer.history.add_epoch(train_loss, valid_loss, score)
            
            scheduler.step()
            
            print(f"Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f} | R²: {score:.4f}")
            
            if score > best_score:
                best_score = score
                self._save_model(model, fold)
                print("✨ R² score improved! Saving model")
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                print(f"   No improvement. Patience: {epochs_no_improve}/{patience}")
            
            # Check for Early Stopping
            if epochs_no_improve >= patience:
                print(f"\n🛑 Early stopping triggered after {patience} epochs without improvement.")
                break
   
        # Store fold score
        self.fold_scores[fold] = best_score
        
        # Generate visualizations for this fold
        print(f"\n📊 Generating visualizations for Fold {fold}...")
        self.visualizer.plot_learning_curves(trainer.history, fold, self.config.freeze_epochs)
        self.visualizer.plot_prediction_scatter(trainer.history, fold, self.scorer)
        
        # Finish
        elapsed = (time.time() - start_time) / 60
        print(f"\n🎉 Fold {fold} completed ({elapsed:.2f} minutes)")
        print(f"Best R² score: {best_score:.4f}")
        
        # Free memory
        del model, train_loader, valid_loader
        gc.collect()
        torch.cuda.empty_cache()
    
    def _freeze_backbone(self, model: nn.Module) -> None:
        """Freeze backbone parameters"""
        backbone = model.module.backbone if isinstance(model, nn.DataParallel) else model.backbone
        for param in backbone.parameters():
            param.requires_grad = False
    
    def _unfreeze_backbone(self, model: nn.Module) -> None:
        """Unfreeze backbone parameters"""
        backbone = model.module.backbone if isinstance(model, nn.DataParallel) else model.backbone
        for param in backbone.parameters():
            param.requires_grad = True
    
    def _save_model(self, model: nn.Module, fold: int) -> None:
        """Save model to output directory"""
        state_dict = model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict()
        save_path = self.config.output_dir / f'best_model_fold{fold}.pth'
        torch.save(state_dict, save_path)
    
    def run_all_folds(self) -> None:
        """Execute training for all folds and generate summary visualization"""
        # Prepare data
        self.data_prep.load_and_pivot()
        self.data_prep.create_stratified_folds(self.data_prep.df_wide)
        
        # Train each fold
        for fold in range(self.config.n_folds):
            try:
                self.run_fold(fold)
            except Exception as e:
                print(f"\n❌ Error occurred in Fold {fold}: {e}")
                gc.collect()
                torch.cuda.empty_cache()
                raise
        
        # Generate fold comparison plot
        print(f"\n{'='*70}")
        print("📊 Generating cross-validation summary...")
        print(f"{'='*70}")
        self.visualizer.plot_fold_comparison(self.fold_scores)


# ============================================================================
# Main Execution
# ============================================================================

if __name__ == '__main__':
    # Initialize configuration
    config = Config()
    config.display_info()
    
    # Execute pipeline
    pipeline = TwoStageTrainingPipeline(config)
    pipeline.run_all_folds()
    
    print("\n" + "="*70)
    print("🎊 All Fold Training Completed!")
    print("="*70)

Configuration
Device: cuda
Backbone: eva02_base_patch14_448.mim_in22k_ft_in1k
Image Size: 448x448
Batch Size: 12
Total Epochs: 50
  - Stage 1 (Freeze): Epoch 1-20 (LR=0.0003)
  - Stage 2 (Finetune): Epoch 21-50 (LR=3e-06)
Cross-Validation: 5-Fold
Output Directory: baseline

Loading CSV: csiro-biomass/train.csv
Long format: 1785 rows
Wide format: 357 rows × 13 columns

First 5 rows:
  sample_id_prefix              image_path Sampling_Date State  \
0     ID1011485656  train/ID1011485656.jpg      2015/9/4   Tas   
1     ID1012260530  train/ID1012260530.jpg      2015/4/1   NSW   
2     ID1025234388  train/ID1025234388.jpg      2015/9/1    WA   
3     ID1028611175  train/ID1028611175.jpg     2015/5/18   Tas   
4     ID1035947949  train/ID1035947949.jpg     2015/9/11   Tas   

             Species  Pre_GSHH_NDVI  Height_Ave_cm  Dry_Clover_g  Dry_Dead_g  \
0    Ryegrass_Clover           0.62         4.6667        0.0000     31.9984   
1            Lucerne           0.55        16.0000        

/tmp/ipykernel_2241608/143795945.py:146: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_wide = df_long.groupby(cols).apply(lambda df: df.set_index('target_name').target)
/tmp/ipykernel_2241608/143795945.py:240: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_stats = df.groupby('Sampling_Date').apply(



🚀 Starting Fold 0 Training (Two-Stage)
Training data: 287 images | Validation data: 70 images

--- Stage 1: Backbone Frozen (Epoch 1-20) ---

Epoch 1/50 (Stage 1)



Target          | R² Score  
----------------------------
Dry_Green       | 0.0121
Dry_Dead        | -0.1053
Dry_Clover      | -0.0664
GDM             | -0.0885
Total           | -0.1651
----------------------------
Weighted Avg    | -0.1162

Train Loss: 21.4687 | Valid Loss: 15.4190 | R²: -0.1162
✨ R² score improved! Saving model

Epoch 2/50 (Stage 1)



Target          | R² Score  
----------------------------
Dry_Green       | 0.1181
Dry_Dead        | 0.0115
Dry_Clover      | -0.0395
GDM             | 0.4566
Total           | 0.3800
----------------------------
Weighted Avg    | 0.2903

Train Loss: 13.2473 | Valid Loss: 11.6218 | R²: 0.2903
✨ R² score improved! Saving model

Epoch 3/50 (Stage 1)


...


Target          | R² Score  
----------------------------
Dry_Green       | 0.6901
Dry_Dead        | 0.4606
Dry_Clover      | -0.9459
GDM             | 0.7181
Total           | 0.6646
----------------------------
Weighted Avg    | 0.4964

Train Loss: 6.3821 | Valid Loss: 7.5796 | R²: 0.4964
✨ R² score improved! Saving model

Epoch 10/50 (Stage 1)



Target          | R² Score  
----------------------------
Dry_Green       | 0.6668
Dry_Dead        | 0.4811
Dry_Clover      | -1.5873
GDM             | 0.7363
Total           | 0.6586
----------------------------
Weighted Avg    | 0.4326

Train Loss: 5.9230 | Valid Loss: 7.6501 | R²: 0.4326

Epoch 11/50 (Stage 1)



Target          | R² Score  
----------------------------
Dry_Green       | 0.5797
Dry_Dead        | 0.2051
Dry_Clover      | -0.4608
GDM             | 0.6192
Total           | 0.4995
----------------------------
Weighted Avg    | 0.4060

Train Loss: 5.6325 | Valid Loss: 8.8774 | R²: 0.4060

Epoch 12/50 (Stage 1)



Target          | R² Score  
----------------------------
Dry_Green       | 0.7503
Dry_Dead        | 0.1112
Dry_Clover      | -0.3379
GDM             | 0.7892
Total           | 0.6015
----------------------------
Weighted Avg    | 0.5109

Train Loss: 5.9660 | Valid Loss: 7.2302 | R²: 0.5109
✨ R² score improved! Saving model

Epoch 13/50 (Stage 1)


KeyboardInterrupt: 

In [ ]:
#0.6497 = do_swap